In [1]:
%pip install duckdb duckdb-engine==0.15.0 jupysql==0.11.0

In [2]:
from google.colab import drive
from google.colab import auth
from google.auth import default
import gspread
import duckdb
import pandas as pd

drive.mount('/content/drive')
%load_ext sql

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [4]:
spreadsheet = gc.open("mart_tables")
fct_citibike_rides_by_year = spreadsheet.worksheet('fct_citibike_rides_by_year')
dim_citibike_stations = spreadsheet.worksheet('dim_citibike_stations')

In [5]:
df_fct_citibike_rides_by_year = None
df_dim_citibike_stations = None

with duckdb.connect('/content/drive/MyDrive/citibike_analysis/dev.duckdb') as conn:
    df_fct_citibike_rides_by_year = conn.sql('select * from fct_citibike_rides_by_year').df()
    df_dim_citibike_stations = conn.sql('select * from dim_citibike_stations').df()

    fct_citibike_rides_by_year.update([df_fct_citibike_rides_by_year.columns.values.tolist()] + df_fct_citibike_rides_by_year.fillna(-1).values.tolist())
    dim_citibike_stations.update([df_dim_citibike_stations.columns.values.tolist()] + df_dim_citibike_stations.fillna(-1).values.tolist())